In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from RobotModel import RobotDynamicsModel, TransformerDecoderModel2D


# Ensure matplotlib inline mode for Jupyter
%matplotlib inline

In [2]:
# file_paths = ["../RC/Data/Mujoco_less_friction_track_mppi.csv",
#               "../RC/Data/Mujoco_less_friction_track_mppi2.csv",
#               "../RC/Data/Mujoco_less_friction_track_manual.csv",
#               "../RC/Data/Mujoco_less_friction_track_manual2.csv",
#               "../RC/Data/Mujoco_less_friction_track_manual3.csv",
#               "../RC/Data/Mujoco_less_friction_track_manual4.csv",
#               "../RC/Data/Mujoco_less_friction_track_manual5.csv",
#               "../RC/Data/Mujoco_less_friction_track_manual6.csv",
#               "../RC/Data/Mujoco_less_friction_track_NNmppi.csv",
#               "../RC/Data/Mujoco_less_friction_track_NNmppi2.csv",
#               "../RC/Data/Mujoco_less_friction_track_NNmppi3.csv",
#               "../RC/Data/Mujoco_less_friction_random.csv",
#               "../RC/Data/Mujoco_less_friction_random2.csv",
#               "../RC/Data/Mujoco_less_friction_random3.csv",
#               "../RC/Data/Mujoco_less_friction_random4.csv",
#               "../RC/Data/Mujoco_less_friction_random5.csv"]

file_paths = ["../RC/Data/Mujoco_less_friction_random5.csv"]

# file_paths = ["Data/Mujoco_less_friction_track_manual.csv", "Data/Mujoco_less_friction_track_manual2.csv", "Data/Mujoco_less_friction_track_manual3.csv", "Data/Mujoco_less_friction_track_manual4.csv", "Data/Mujoco_less_friction_track_manual5.csv", "Data/Mujoco_less_friction_track_manual6.csv"]
# file_paths = ["Data/Mujoco_less_friction_track_mppi.csv", "Data/Mujoco_less_friction_track_mppi2.csv"]

In [ ]:
cur_time = pd.Timestamp.now()
tag = f"{cur_time.year}_{cur_time.month}_{cur_time.day}_{cur_time.hour}_{cur_time.minute}"

In [3]:
def plot_path(states):
    x = states[:,0]
    y = states[:,1]
    yaw = states[:, 2]
    # Create a figure for the plot
    plt.figure(figsize=(10, 10))

    # Plot the robot's trajectory
    plt.plot(x, y, label="Trajectory")

    # Add quiver plot to show yaw at different points
    skip = 1  # Plot yaw every 'skip' points to avoid clutter
    plt.quiver(x[::skip], y[::skip], np.cos(yaw[::skip]), np.sin(yaw[::skip]), 
            color='r', scale=50, label="Yaw")

    # Add labels and title
    plt.xlabel("X position")
    plt.ylabel("Y position")
    plt.title("Robot Pose (Trajectory and Yaw)")
    plt.legend()

    # Show the plot
    plt.grid(True)
    plt.axis('equal')  # Ensure equal scaling for both axes
    plt.show()

In [4]:
import sophuspy as sp
def pi2pi(angle):
    """
    Normalize angle to [-π, π] range using modulo operation.
    
    Args:
        angle (float): Input angle in radians
    
    Returns:
        float: Normalized angle in radians between -π and π
    """
    return ((angle + np.pi) % (2 * np.pi)) - np.pi

def rotation_matrix_SO2(yaw):
    return np.array([[np.cos(yaw), - np.sin(yaw)], [np.sin(yaw), np.cos(yaw)]])
    
# Assume sophuspy provides SE2 for pose manipulation and log/exp mapping
def apply_relative_transformation(states):
    """Transform all states with respect to the last pose using SE(2) log and exp."""
    transformed_states = []
    
    pivot_pose = sp.SE2(rotation_matrix_SO2(states[-2, 2]), states[-2, 0:2])
    
    # Transform each pose relative to the last one
    for x, y, yaw in states:
        current_pose = sp.SE2(rotation_matrix_SO2(yaw), np.array([x, y]))
        relative_pose = pivot_pose.inverse() * current_pose  # relative transformation
        # print(relative_pose.log())
        transformed_states.append( (relative_pose).log())
    return transformed_states

def apply_simple_deletion_transformation(states):
    transformed_states = []
    pivot_pose = states[-2]
    for i in range(len(states)):
        transformed_states.append(states[i] - pivot_pose)
    return transformed_states


In [5]:
# Constants
h = 20  # history length
batch_size = 500 # Define batch size for DataLoader
train_val_split_ratio = 0.8  # 80% train, 20% validation
state_columns = ['x', 'y', 'yaw']
states_dof = len(state_columns)  # x, y, yaw
# Prepare the data
action_columns = ['accel', 'steer']

# Create sequences with history length h
input_sequences = []
output_sequences = []
tot_duration = 0
num_of_augmentation = 1
# file_paths = ["Data/Mujoco_less_friction_track_mppi.csv"]
for file_path in file_paths:
    path_data = pd.read_csv(file_path)

    # Display the first few rows
    path_data.head()
    times = np.array(path_data["timestamp"])
    dataset_duration = times[-1] - times[0]
    tot_duration += dataset_duration
    print(f"{file_path} is {dataset_duration/60}m")
    states = path_data[state_columns].values
    actions = path_data[action_columns].values
    # plot_path(states)
    # cnt = 0
    for _ in range(num_of_augmentation):
        # plot_path(augmented_states)
        for i in range(len(states) - h - 20 * 30):
            # Original sequence
            # plot_path(states[i:i + h + 1])
            if times[i+h] - times[i+h - 1] > 1/18:
                # cnt+=1
                continue

            transformed_state = apply_relative_transformation(states[i:i + h + 1])
            # transformed_state = states[i:i + h + 1]
            # plot_path(np.array(transformed_state))
            # break
            # plot_path(np.array(transformed_state))
            input_seq = np.concatenate((transformed_state[:h], actions[i:i + h]), axis=1).flatten()
            output_seq = transformed_state[h]
            input_sequences.append(input_seq)
            output_sequences.append(output_seq)
    # print(len(times) -  cnt)
print(f"Total duration is {tot_duration/60}m")


../RC/Data/Mujoco_less_friction_random5.csv is 69.79429014921189m
Total duration is 69.79429014921189m


In [6]:
# Convert to NumPy arrays and then to PyTorch tensors
input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)

# Convert to PyTorch tensors
inputs = torch.tensor(input_sequences, dtype=torch.float32)
outputs = torch.tensor(output_sequences, dtype=torch.float32)
# Check shapes
print(inputs.shape, outputs.shape)  # (samples, h * (|x| + |a|)), (samples, |x|)

# Dataset and DataLoader Setup
dataset = TensorDataset(inputs, outputs)

# Split dataset into train, validation, and test sets (80-10-10 split)
train_size = int(len(dataset) * train_val_split_ratio)
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

torch.Size([83133, 100]) torch.Size([83133, 3])


In [7]:

# Instantiate the model
input_size = h * (len(state_columns) + len(action_columns))  # h * (|x| + |a|)

# model = RobotDynamicsModel(input_size, len(state_columns))
# model

model = TransformerDecoderModel2D(input_size, len(state_columns))
model

TransformerDecoderModel2D(
  (input_projection): Linear(in_features=100, out_features=128, bias=True)
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dr

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define the evaluate function to calculate loss on validation/test sets
def evaluate(loader):
    model.eval()  # Set model to evaluation mode
    total_loss, total_samples = 0, 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            predictions = model(inputs)
            loss = criterion(predictions, targets)
            total_loss += loss.item() * len(targets)
            total_samples += len(targets)

    return total_loss / total_samples  # Return average loss

In [ ]:
print(f"Training w {device}")
# Training loop with validation
best_val_loss = float('inf')  # Initialize best validation loss to infinity
train_losses, val_losses = [], []
epochs = 100
last_saved_epoch = - 10
for epoch in range(epochs):
    model.train()  # Set model to training mode

    train_loss = 0  # Accumulate training loss
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # print(inputs.shape)

        # Forward pass
        optimizer.zero_grad()  # Clear previous gradients
        predictions = model(inputs)
        # print(predictions.shape, targets.shape)
        loss = criterion(predictions, targets)  # Compute loss

        # Backward pass and optimization
        loss.backward()  # Backpropagate gradients
        optimizer.step()  # Update model parameters

        train_loss += loss.item() * len(targets)  # Accumulate loss

    # Calculate average train loss for the epoch
    train_loss /= len(train_set)

    # Evaluate on validation set
    val_loss = evaluate(val_loader)
    # Track losses per epoch
    if val_loss < 10 and train_loss < 10:
        train_losses.append(train_loss)
        val_losses.append(val_loss)

    # Save the best model based on validation loss
    if val_loss < best_val_loss and epoch - last_saved_epoch > 10:
        last_saved_epoch = epoch
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'Mujoco_model_v2.pth')
        print(f"Saved Best Model at Epoch {epoch + 1} with Validation Loss: {val_loss:.4f}")

    # Print progress every 1000 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Evaluate on the test set after training completes
test_loss = evaluate(test_loader)
print(f'Test Loss: {test_loss:.4f}')

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()
plt.show()